In [4]:
import selenium
import time
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.common import exceptions

In [5]:
driver = webdriver.Chrome()
driver.get('https://sellercentral.amazon.com/forums/c/amazon-custom')

In [6]:
for i in range(30):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

data = driver.find_elements_by_css_selector('a[class = "title raw-link raw-topic-link"]')
titles = [x.text for x in data]
links = [x.get_attribute('href') for x in data]

subcat_data = driver.find_elements_by_css_selector('span[class = "category-name"]')
subcategory = [x.text for x in subcat_data[2:]]

contents = []
for link in links:
    driver.get(link)
    data = driver.find_elements_by_css_selector('div[class = "cooked"]')
    content = [x.text.replace('\n', ' ') for x in data]
    contents.append(content)

time.sleep(5)
driver.quit()

In [7]:
# Taking Posts Content
posts = []
responses = []
for line in contents:
    posts.append(line[0])
    responses.append(line[1:])

In [22]:
def response_df(urls, resps):
    column_names = ['URL', 'Response Text']
    df_resps = pd.DataFrame(columns = column_names)
    for url,line in zip(urls, resps):
        n = len(line)
        sub_urls = [url] * n
        new_df = pd.DataFrame(list(zip(sub_urls, line)), columns = column_names)
        df_resps = pd.concat([df_resps, new_df])
    return df_resps

In [25]:
DF1 = pd.DataFrame(titles, columns=['Title'])
DF1['Category'] = 'Amazon Custom'
DF1['Sub-category'] = subcategory
DF1['Post Text'] = posts
DF1['URL'] = links

In [26]:
DF1.head()

,Title,Category,Sub-category,Post Text,URL
0,Testing requirements in toys and games category,Amazon Custom,Amazon Custom Forum,My product is domino sets for adults. I am the...,https://sellercentral.amazon.com/forums/t/test...
1,Food and Drug Administration,Amazon Custom,Amazon Custom,"Hello, I am on my way to becoming an Amazon de...",https://sellercentral.amazon.com/forums/t/food...
2,Images Storage for mass upload,Amazon Custom,Amazon Custom,When uploading products with the spreadsheet w...,https://sellercentral.amazon.com/forums/t/imag...
3,Prepaid Return Label program email clarification,Amazon Custom,Amazon Custom Forum,You may have received an email from Amazon inf...,https://sellercentral.amazon.com/forums/t/prep...
4,Pulling Amazon Custom Personalizatoin Data fro...,Amazon Custom,Amazon Custom Forum,The problem: Currently Amazon gives you links ...,https://sellercentral.amazon.com/forums/t/pull...


In [27]:
DF2 = response_df(links, responses)

In [28]:
DF2.head()

,URL,Response Text
0,https://sellercentral.amazon.com/forums/t/imag...,I’m confused. The spreadsheet simply points to...
0,https://sellercentral.amazon.com/forums/t/prep...,Will I have the ability to add my RMA number t...
1,https://sellercentral.amazon.com/forums/t/prep...,The option will remain as it currently is
2,https://sellercentral.amazon.com/forums/t/prep...,Were based in the UK and sell mostly low value...
0,https://sellercentral.amazon.com/forums/t/pull...,We have the same issue and are very surprised ...


In [29]:
DF1.describe

<bound method NDFrame.describe of                                                  Title       Category  \
0      Testing requirements in toys and games category  Amazon Custom   
1                         Food and Drug Administration  Amazon Custom   
2                       Images Storage for mass upload  Amazon Custom   
3     Prepaid Return Label program email clarification  Amazon Custom   
4    Pulling Amazon Custom Personalizatoin Data fro...  Amazon Custom   
..                                                 ...            ...   
379                              Amazon Custom Program  Amazon Custom   
380  Just enabled ‘Amazon Custom’ on my products, a...  Amazon Custom   
381      Amazon Custom - Refunds. To do, or not to do?  Amazon Custom   
382                              Custom Order Feedback  Amazon Custom   
383  Amazon custom (personalized products) and thir...  Amazon Custom   

            Sub-category                                          Post Text  \
0    Amazo

In [30]:
DF2.describe

<bound method NDFrame.describe of                                                   URL  \
0   https://sellercentral.amazon.com/forums/t/imag...   
0   https://sellercentral.amazon.com/forums/t/prep...   
1   https://sellercentral.amazon.com/forums/t/prep...   
2   https://sellercentral.amazon.com/forums/t/prep...   
0   https://sellercentral.amazon.com/forums/t/pull...   
..                                                ...   
11  https://sellercentral.amazon.com/forums/t/amaz...   
12  https://sellercentral.amazon.com/forums/t/amaz...   
13  https://sellercentral.amazon.com/forums/t/amaz...   
0   https://sellercentral.amazon.com/forums/t/cust...   
1   https://sellercentral.amazon.com/forums/t/cust...   

                                        Response Text  
0   I’m confused. The spreadsheet simply points to...  
0   Will I have the ability to add my RMA number t...  
1           The option will remain as it currently is  
2   Were based in the UK and sell mostly low value...  
0

In [31]:
DF1.to_csv("DF1.csv", index = False)
DF2.to_csv("DF2.csv", index = False)